In [2]:
import os

In [3]:
cd ..

c:\Thesis\Roshan_Projects\end-to-end_ML_with_MLflow


In [4]:
import dagshub
dagshub.init(repo_owner='Roshan-RB', repo_name='end-to-end_ML_with_MLflow', mlflow=True)

Accessing as Roshan-RB

Initialized MLflow to track repo "Roshan-RB/end-to-end_ML_with_MLflow"

Repository Roshan-RB/end-to-end_ML_with_MLflow initialized!

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [14]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Roshan-RB/end-to-end_ML_with_MLflow.mlflow",
           
        )

        return model_evaluation_config


In [16]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [17]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # TRACKING → DagsHub
        mlflow.set_tracking_uri(self.config.mlflow_uri)

        # REGISTRY → Must be local (DagsHub does NOT support registry)
        mlflow.set_registry_uri("file:///tmp/mlruns")

        with mlflow.start_run():

            predicted = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted)

            save_json(Path(self.config.metric_file_name),
                    {"rmse": rmse, "mae": mae, "r2": r2})

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Save model using joblib (NO registry involved)
            local_model_path = "artifacts/model_evaluation/model.joblib"
            joblib.dump(model, local_model_path)

            # Log the joblib model file as a regular artifact
            mlflow.log_artifact(local_model_path)


        print("Logged successfully to DagsHub MLflow!")


In [18]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-11-24 15:42:21,789: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-24 15:42:21,791: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-24 15:42:21,793: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-24 15:42:21,794: INFO: common: created directory at: artifacts]
[2025-11-24 15:42:21,795: INFO: common: created directory at: artifacts/model_evaluation]
[2025-11-24 15:42:22,534: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run clumsy-newt-441 at: https://dagshub.com/Roshan-RB/end-to-end_ML_with_MLflow.mlflow/#/experiments/0/runs/5f48730b2f8640faa873da01a95437ef
🧪 View experiment at: https://dagshub.com/Roshan-RB/end-to-end_ML_with_MLflow.mlflow/#/experiments/0
Logged successfully to DagsHub MLflow!
